In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import re # regular expressions (pattern matching and searching)

# nltk : Natural Language Toolkit
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer # for stemming (gives root word)
# TF-IDF (Term Frequency-Inverse Document Frequency) 
from sklearn.feature_extraction.text import TfidfVectorizer # convert text to numerical feature vector

from sklearn.svm import SVC # Support Vector Classifier (SVC) 

In [3]:
# import nltk
# nltk.download('stopwords') # downloading all the stopwords if not already present

In [4]:
print(stopwords.words('english')) # words that dosen't add much value to data set

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Pre-processing

loading the Data Set

In [5]:
train = pd.read_csv('/kaggle/input/fake-news/train.csv')
test = pd.read_csv('/kaggle/input/fake-news/test.csv')

In [6]:
train.shape # number of rows and cloumns

(20800, 5)

In [7]:
test.shape

(5200, 4)

In [8]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [10]:
train.isnull().sum() # checking the missing values

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [12]:
train = train.fillna('') #replacing null values with empty string
test = test.fillna('')

In [13]:
train.isnull().sum() # no null values

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [14]:
test.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

Merging (author) and (title) into new column and storing it in X : 

In [15]:
X_train = train['author'] + ' ' + train['title']
X_test = test['author'] + ' ' + test['title']

In [16]:
X_train.head()

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
dtype: object

In [17]:
X_test.head()

0    David Streitfeld Specter of Trump Loosens Tong...
1     Russian warships ready to strike terrorists n...
2    Common Dreams #NoDAPL: Native American Leaders...
3    Daniel Victor Tim Tebow Will Attempt Another C...
4    Truth Broadcast Network Keiser Report: Meme Wa...
dtype: object

Seperating label column from data set

In [18]:
Y_train = train['label'] # storing label in Y

In [19]:
Y_train.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

## Stemming

Reducing words (prefix & suffix) to its root word

In [20]:
port_stem = PorterStemmer() # creating instance

In [21]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # replace anything other than a_z with ' '
    stemmed_content = stemmed_content.lower() # convert to lower case 
    stemmed_content = stemmed_content.split() # split all the words to list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # stemming + removing stopwords
    stemmed_content = ' '.join(stemmed_content) # again joining all words with ' '
    return stemmed_content

Applying stemming on X

In [22]:
X_train = X_train.apply(stemming)
X_test = X_test.apply(stemming)

In [23]:
X_train.head()

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
dtype: object

In [24]:
X_test.head()

0    david streitfeld specter trump loosen tongu pu...
1    russian warship readi strike terrorist near al...
2    common dream nodapl nativ american leader vow ...
3    daniel victor tim tebow attempt anoth comeback...
4     truth broadcast network keiser report meme war e
dtype: object

separating the data and label (into array format)

In [25]:
X_train = X_train.values # Data Array
Y_train = Y_train.values # Label
X_test = X_test.values

In [26]:
print(X_train)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [27]:
print(X_test)

['david streitfeld specter trump loosen tongu purs string silicon valley new york time'
 'russian warship readi strike terrorist near aleppo'
 'common dream nodapl nativ american leader vow stay winter file lawsuit polic'
 ... 'mike mcphate california today exactli sushi new york time'
 'us marin deploy russian border norway'
 'teddi wayn awkward sex onscreen new york time']


In [28]:
print(Y_train)

[1 0 1 ... 0 1 1]


In [29]:
X_train.shape

(20800,)

In [30]:
X_test.shape

(5200,)

In [31]:
Y_train.shape

(20800,)

## Vectorizing (Textual data to numerical data)

In [32]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train) # converting into feature vector

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [33]:
print(X_train)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [34]:
print(X_test)

  (0, 16996)	0.08864595251126647
  (0, 16088)	0.3205594987716609
  (0, 15582)	0.11032513315099028
  (0, 15368)	0.38547947461491155
  (0, 15295)	0.08697876504498342
  (0, 14605)	0.3970837204511862
  (0, 14591)	0.34467447913058513
  (0, 13850)	0.32956045112215354
  (0, 12030)	0.38547947461491155
  (0, 10306)	0.08568694729690247
  (0, 8842)	0.36912420870480717
  (0, 3623)	0.209228842739656
  (1, 16473)	0.4921981982038152
  (1, 15142)	0.3550835494981517
  (1, 14603)	0.35559908473568963
  (1, 13048)	0.3052969733971102
  (1, 12300)	0.3838422559012773
  (1, 10219)	0.37818137822242154
  (1, 347)	0.3487744590431239
  (2, 16746)	0.3016628915147435
  (2, 16354)	0.2909537422615255
  (2, 14457)	0.30637021801826614
  (2, 11598)	0.21447077225895875
  (2, 10419)	0.35982822390508096
  (2, 10176)	0.3194727535249622
  :	:
  (5196, 5884)	0.2500401085058591
  (5196, 1517)	0.19940396943572683
  (5196, 41)	0.2656503884474958
  (5197, 16996)	0.11797520134546996
  (5197, 15336)	0.33413616793311524
  (5197, 152

In [35]:
print(X_train.shape)
print(X_test.shape)

(20800, 17128)
(5200, 17128)


# SVM (SUPPORT VECTOR MACHINE):

In [36]:
classifier = SVC(kernel="linear") # we need a linear decision boundary

In [37]:
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [38]:
X_predict = classifier.predict(X_test)

In [39]:
X_predict

array([0, 1, 1, ..., 0, 1, 0])

In [40]:
output = pd.DataFrame({'id': test.id, 'label': X_predict})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
